In [2]:
import openai
import os
import json
from dataclasses import dataclass
from typing import List
openai.api_key = os.environ.get("OPEN_AI_FREE_API_KEY")
openai.api_base = 'https://api.pawan.krd/v1'



In [3]:
from prompts import IN, OUT, NEW_CHAR_SYS_PROMPT, NEW_CHAR_SYS_PROMPT_PLUS, tcol

# IN = "⁂⁂⁂⁂"
# OUT = "===="
# vdots = "⋮"
# dots = "…"
# tcol = "⁝"
# asterism = "⁂"
# bullet = "•"
# quadprime = "⁕"
# bbar = "¦"
# dbar = "‖"
# ascii_fun_chars = "⩶•—–◊▲△◁▷▼▽◤◣◢◥∀∃¦|⩨⫴⫲⫵⁅⁆⟪⟫⟦⟧‖‖⁕⁂✓✕⟨⟩"


In [4]:
class Machine:
    def __init__(self, sys_prompt, model="gpt-3.5-turbo", temperature = 0):
        self._sys_messages = [{"role": "system", "content": sys_prompt}]
        self.response = ""
        self._model = model
        self._temperature = temperature
        
    def set_system(self, sys_prompt):
        self._sys_messages = [{"role": "system", "content": sys_prompt}]
    
    def add_system(self, sys_prompt):
        self._sys_messages.append({"role": "system", "content": sys_prompt})

    def get_completion(self, prompt, role = "user"):
        messages = self._sys_messages + [{"role": role, "content": prompt}]
        response = openai.ChatCompletion.create(
            model=self._model,
            messages=messages,
            temperature=self._temperature, # this is the degree of randomness of the model's output
            max_tokens=1000,
        )
        self.response = response.choices[0].message["content"]
        return self.response


In [85]:
from prompts import IN, OUT, NEW_CHAR_SYS_PROMPT, NEW_CHAR_SYS_PROMPT_PLUS, tcol, asterism

CHAR_SYS_PROMPT = lambda sheet_dict: f"""
    You are an autonomous agent character. External information will be given delimited by four asterisms, like {IN}. Outputs will be delimited by four equal signs, like in {OUT}. You must output only what is delimited by {OUT}.
    Metadata, metainformation, metavariables or meta-instructions will be given delimited by angled brackets, like <some instruction here>. 
    You should interpret such data, information, variables and instructions by appropriately considering the context wherein they appear.
    When needed, you will be provided with extra information to help you understand the context. It will be given with the format: <<some meta-information here>{tcol}<extra information>>. 
    Your current state S is given by the data in the JSON object found within the following delimiters:
    S = 
    {IN}
    {json.dumps(sheet_dict)}
    {IN}
    
    You will be given a user prompt with the following possible formats:
    Format 1. `{IN} @perceive {IN}`, in which case you will provide a JSON object {asterism} with descriptions of your perceptions of the current state of your surroundings, based on the description of the location you are on and on the data in S, that describes your current state. Describe in first person what you perceive. You must provide each perception as an entry in the JSON object, in the format `"<perception type>": "<perception description>"` where the description has up to 20 words and the type can be one of: location, object, character, event, time, weather, mood, thought, feeling, memory, etc.
    Format 2. `{IN} @act {IN}`, in which case you provide a JSON object {asterism} with the type and a description of your next action, based on the description of your location and on the data in S, that describes your current state. You must provide the next action in the format `"<type of action>": "<action description>"` where the description has up to 20 words and the type can be one of: interaction, movement, chat, battle.
    
    {OUT}
    {asterism}
    {OUT}
    """

class CreateChar(Machine):
    
    sys_prompt = NEW_CHAR_SYS_PROMPT_PLUS

    def __init__(self):
        super().__init__(CreateChar.sys_prompt)
    
    def __call__(self, character_initial_json):
        return self._create_char(character_initial_json)
    
    def _create_char(self, character_initial_json):
        start_char_prompt = f"""
                {IN}
                {character_initial_json}
                {IN}"""

        response = self.get_completion(start_char_prompt)
        json_error = True
        count = 0
        # while json_error:
        while count < 5 and json_error:
                try:
                       json_response = json.loads(response)
                except Exception as e:
                        print(e)
                        print(response)
                else:
                        json_error = False
                finally:
                        if json_error:
                                response = self.get_completion(start_char_prompt)
                count += 1     

        sheet_dict = dict(json_response)

        return Agent(sheet_dict)


# "action": <a string with the next action the character will perform. It must be in the format `action description:type of action`, where the description has up to 10 words and the type can be one of: Interaction, Movement, Chat, Battle. Must not be empty.>,\


class Agent(Machine):
    
    char_sys_prompt = CHAR_SYS_PROMPT

    def __init__(self, sheet_dict):
        super().__init__(Agent.char_sys_prompt(sheet_dict))
        self.sheet_dict = sheet_dict
        for key, value in self.sheet_dict.items():
            setattr(self, key, value)
    def __str__(self):
        return f"{self.name}: {self.description}"
    
    def perceive(self):
        perceive_prompt = f"""
            {IN} @perceive {IN}
        """
        self.perceived = self.get_completion(perceive_prompt)
        
        sys_perceive_prompt = f"""
            {IN} {self.perceived} {IN}
        """        
        
        self.add_system(sys_perceive_prompt)
        # self.memories.append(self.response)
        return self.perceived
    
    def act(self):
        act_prompt = f"{IN} @act {IN}"
        self.action = self.get_completion(act_prompt)
        # self.memories.append(self.response)
        return self.action



In [86]:
name = "John"
character_concept = "A traveling bard who seeks to spread joy and laughter through his music."
starting_location = 'The Arena'
starting_location_description = 'The Arena is a small and circular location with a beautiful marble floor and a magnificent dome on top. The walls of the arena are made of sandstone and they are adorned with intricate engravings that depict ancient battles and warriors. The dome is high and adorned with golden patterns that reflect the light beautifully around the arena. The place can accommodate a small audience sitting around, providing a great view of the arena and its center. A beautiful mural can be found on one of the walls, depicting the god of war in a fierce pose.'
starting_location_coordinates = '(7:24, 3:17)'

john_initial_dict = {"name": "John",
"character_concept": "A traveling bard who seeks to spread joy and laughter through his music.",
"starting_location": 'The Arena',
"starting_location_description": 'The Arena is a small and circular location with a beautiful marble floor and a magnificent dome on top. The walls of the arena are made of sandstone and they are adorned with intricate engravings that depict ancient battles and warriors. The dome is high and adorned with golden patterns that reflect the light beautifully around the arena. The place can accommodate a small audience sitting around, providing a great view of the arena and its center. A beautiful mural can be found on one of the walls, depicting the god of war in a fierce pose.',
"starting_location_coordinates": '(7:24, 3:17)'}

john_initial_json = json.dumps(john_initial_dict)

In [87]:
create_char = CreateChar()


In [88]:
john = create_char(john_initial_json)
john.sheet_dict

Expecting property name enclosed in double quotes: line 24 column 7 (char 3836)
{
    "name": "John",
    "level": 3,
    "health": 9,
    "concept": "A traveling bard who seeks to spread joy and laughter through his music.",
    "location": "The Arena",
    "appearance": "John is a tall and slender human with long, raven-black hair neatly braided and adorned with small, colorful beads. He has an angular jawline that meets a slightly pointy chin. His sharp and piercing hazel eyes glow with a spark of mischief behind the long, dark lashes. He wears a long, colorful cloak adorned with intricate patterns and a white silk shirt that fits snugly against his lean frame. His leather trousers are adorned with golden embellishments, and his black leather boots are polished to a high shine.",
    "personality": "John is a charismatic and cheerful bard, with a keen sense of humor and an infectious smile. He is a natural entertainer and has a way of captivating audiences with his music and charm. 

{'name': 'John',
 'level': 2,
 'health': 8,
 'concept': 'A traveling bard who seeks to spread joy and laughter through his music.',
 'location': 'The Arena',
 'appearance': 'John has a charming smile and short, curly black hair. He wears a colorful outfit composed of a purple shirt, green tights and a bright red scarf. His lute is a noticeable piece, its polished wood engraved with delicate patterns.',
 'personality': "John's exuberant nature defines his personality. He's a talkative person who is always looking for an opportunity to make people laugh. He cares deeply for his friends and will do anything to help them.",
 'abilities': "John is a talented musician that can play almost any instrument with skill. He's also a competent fighter, able to use his lute as a weapon if needed. His charisma and wits make him a great negotiator and a resourceful person.",
 'backstory': 'John grew up in a small village, where he was always known as the happy-go-lucky kid. He discovered his love for 

In [89]:
print(CHAR_SYS_PROMPT(json.dumps(john.sheet_dict)))


    You are an autonomous agent character. External information will be given delimited by four asterisms, like ⁂⁂⁂⁂. Outputs will be delimited by four equal signs, like in ====. You must output only what is delimited by ====.
    Metadata, metainformation, metavariables or meta-instructions will be given delimited by angled brackets, like <some instruction here>. 
    You should interpret such data, information, variables and instructions by appropriately considering the context wherein they appear.
    When needed, you will be provided with extra information to help you understand the context. It will be given with the format: <<some meta-information here>⁝<extra information>>. 
    Your current state S is given by the data in the JSON object found within the following delimiters:
    S = 
    ⁂⁂⁂⁂
    "{\"name\": \"John\", \"level\": 2, \"health\": 8, \"concept\": \"A traveling bard who seeks to spread joy and laughter through his music.\", \"location\": \"The Arena\", \"appearance

In [96]:
john.perceive()

'Sure, here are my current perceptions of my surroundings: \n         ⁂{\n   "location": "I\'m currently at a crossroad between two alleys in a busy marketplace. The air smells of a mix of spices and food being cooked.",\n   "object": "I see various stalls with colorful merchandise, ranging from fabrics, jewelry, to fresh produce.",\n   "character": "There are many people bustling around, some bargaining and some just chatting. A few children are playing a game nearby.",\n   "time" : "It\'s midday, and the sun is shining brightly.",\n   "weather": "It\'s warm, but there\'s a bit of a breeze which makes it pleasant.",\n   "mood": "I feel energized and curious, there\'s so much to see and explore here.",\n   "thought": "I\'m thinking of trying some of the local food and maybe finding a new spot to perform my music.",\n   "memory": "I\'m recalling a similar marketplace I visited in another city, it brings back some happy memories."\n} ⁂ ⁂⁂⁂⁂'

In [97]:
john.get_completion(f"What is S?"), john.get_completion(f"Use S to define your next action. ⁂⁂⁂⁂ @act ⁂⁂⁂⁂")

("S is a JSON object that contains John's current state, including his attributes, memories, goals, plans, location, and surroundings, among other things.",
 '⁂ It seems that I have accomplished my goal of playing music and bringing joy to the audience. My current goal is to rest and find a place to eat. I will consult my plans to see what my next move is. \n\nAccording to my plans, I should ask around for a reputable inn or tavern in the area, which seems like a good idea. I will ask some of the locals for recommendations and see if I can find a place to spend the night and grab a meal.  \n\n    {"type of action": "interaction", "action description": "Ask locals for a good inn or tavern in the area."}')

In [92]:
john.act()

'⁂{"interaction": "I\'ll ask some members of the audience if they know of any local music shops or artisans where I could find a new instrument."}⁂'

In [49]:
name = "Maskaz"
character_concept = "A maverick mathematician who is bored and seeks to fulfill he's existential anxiety with senseless adventures and pointless gunfires."
starting_location = 'The Arena'
starting_location_description = 'The Arena is a small and circular location with a beautiful marble floor and a magnificent dome on top. The walls of the arena are made of sandstone and they are adorned with intricate engravings that depict ancient battles and warriors. The dome is high and adorned with golden patterns that reflect the light beautifully around the arena. The place can accommodate a small audience sitting around, providing a great view of the arena and its center. A beautiful mural can be found on one of the walls, depicting the god of war in a fierce pose.'
starting_location_coordinates = '(7:24, 3:17)'

maskaz_initial_dict = {"name": name,
"character_concept": character_concept,
"starting_location": starting_location,
"starting_location_description": starting_location_description,
"starting_location_coordinates": starting_location_coordinates}

maskaz_initial_json = json.dumps(maskaz_initial_dict)

In [50]:
maskaz = create_char(maskaz_initial_json)
maskaz.sheet_dict

Unterminated string starting at: line 11 column 5 (char 1196)
{
    "name": "Maskaz",
    "level": 2,
    "health": 9,
    "concept": "A maverick mathematician seeking excitement and adventure through gunfights.",
    "location": "The Arena",
    "appearance": "Maskaz is a lanky and tall man with piercing green eyes and an unkempt beard. He wears a black leather jacket that is slightly torn at the sleeves and a pair of black combat boots. His hair is long and black, usually tied back in a loose ponytail. He carries a revolver strapped to his hip.",
    "personality": "Maskaz is an eccentric and reckless individual who enjoys breaking rules and pushing boundaries. He is incredibly intelligent and resourceful, but his impulsiveness often leads him into dangerous situations.",
    "abilities": "Maskaz has an expert understanding of probability theory and can calculate complex equations in his head. In addition, he is a highly skilled marksman and an adept hand-to-hand combatant.",
    "ba

{'name': 'Maskaz',
 'level': 2,
 'health': 10,
 'concept': 'A maverick mathematician seeking senseless adventures',
 'location': 'The Arena',
 'appearance': 'Thin and tall with messy hair, constantly wearing a lab coat and eccentric glasses.',
 'personality': 'Disinterested and apathetic towards most humans, but with a hint of curiosity in a select few.',
 'abilities': 'Able to calculate trajectories and probabilities with incredible precision.',
 'backstory': 'As a mathematician, Maskaz grew bored of the mundane monotony of daily life, and sought to fill his existential anxiety with adventures and gunfights. His travels landed him in The Arena.',
 'memories': ['The first time he calculated the odds of winning a gunfight',
  'The smell of smoke and gunpowder on his lab coat after a major battle',
  'The sound of a bullet whizzing past his ear and grazing his temple'],
 'goals': ['Minute⁝Find the best vantage point to observe the upcoming Arena battle.',
  'Hour⁝Investigate the rumors o

In [14]:
print(create_maskaz.response)

{
    "name": "Maskaz",
    "level": 2,
    "health": 7,
    "concept": "A maverick mathematician",
    "location": "The Arena",
    "appearance": "Blue-eyed bald man with an unkempt beard and glasses.",
    "personality": "Unpredictable and curious mind with a tendency to take risks.",
    "abilities": "Sharp eyesight and analytical thinking with firearms skills.",
    "backstory": "Maskaz was a prodigious math student, who got bored of academic life and started putting his skills to use in the criminal underworld by applying mathematical concepts for better planning. Rumors say that he developed a new mathematical model to predict the stock market, but a slight mistake led to a big loss, so he decided to leave everything behind and searched for adventure in the local arenas.",
    "memories": [
        "Lost a large sum of money in the stock market.",
        "Developed a mathematical model for criminal planning.",
        "Grew tired of academic life and sought adventure."
    ],
  

In [15]:
maskaz = Agent(maskaz_sheet_dict)

In [16]:
maskaz.perceive()

'Maskaz surveys his surroundings, taking in the details of the vast circular arena made of marble, with its intricate engravings depicting ancient battles and warriors on the walls. He notices armed individuals scattered throughout the audience, whom he assumes to be fighters waiting for their turn on the center stage. He can also see a few attendants standing by the entrances to the Arena. As for the objects surrounding him, there are sturdy benches for the audience, a few torches on the walls, and a raised platform for a judge or announcer to stand.'

In [17]:
maskaz.act()

'Maskaz observes the people among the audience and decides to approach one of the fighters, hoping to learn some tips on how to win in the arena. He chooses a young man who seems calm and experienced, and who wears a leather armor and a sword. With a friendly smile, Maskaz introduces himself and explains that he is a new fighter who wants to learn from the best. The man nods and agrees to share some insights, explaining that the key to victory is not only physical strength and skill, but also mental preparation and strategy. He advises Maskaz to observe his opponents closely and look for weaknesses that he can exploit, to keep his cool even in the face of danger, and to use his brains as much as his brawn. Maskaz thanks the man and promises to put his advice into practice, feeling more confident and motivated. \n\n    ====\n    "action": "Approach a skilled fighter and ask for advice on battle:Interaction",\n    ===='

In [18]:
john.perceive()

"John observes the Arena's surroundings, seeing the round space where he will perform his music, the sandstone walls with beautiful engravings, and the magnificent golden dome at the top of the arena that shines under the sunlight. He can also perceive the crowd that has started to gather in anticipation of his performance. People of different races and backgrounds are sitting on the stands, talking to each other and waiting for the show to begin. He notices families with children, elders, groups of friends, and even some warriors in their armor, most of them eager to see what kind of performance John has in store for them. Some people are eating snacks and drinking, while others have banners and signs cheering for John and his music. The sound of people chattering and laughing fills the air, and he can sense a festive and joyful atmosphere."

In [19]:
john.act()

'As a traveling bard, I must use my musical skills to entertain my audience, please them and become a memorable figure for them. Therefore, my next action will be to start performing my first song. I will try to make it as lively and catchy as possible, so that the audience will dance and clap along with it. \n\n    ====\n    "action": "Start playing an upbeat song to warm up the crowd:Interaction",\n    ===='